<a href="https://colab.research.google.com/github/Adarsh2shark/NLP-code-/blob/main/NLP_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import nltk

In [2]:

from google.colab import files
 
 
uploaded = files.upload()

Saving spam.csv to spam.csv


In [3]:

import pandas as pd
import io
 
data = pd.read_csv(io.BytesIO(uploaded['spam.csv']), encoding = 'latin-1')
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [4]:
# dropping irrelevant columns
data = data.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis =1)
data.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
# renaming the column for understanding
data.rename(columns = {"v1" :"label", "v2": "Message"},  inplace = True)
data.head()

,label,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
#######################################################################################
#                           Handling Categorical Data

In [7]:
data = pd.get_dummies(data, columns= ['label'])
data.head()
#  depending on the existence they are marked as 1 or 0

,Message,label_ham,label_spam
0,"Go until jurong point, crazy.. Available only ...",1,0
1,Ok lar... Joking wif u oni...,1,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,0,1
3,U dun say so early hor... U c already then say...,1,0
4,"Nah I don't think he goes to usf, he lives aro...",1,0


In [8]:
data['label_ham'].value_counts()

1    4825
0     747
Name: label_ham, dtype: int64

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Message     5572 non-null   object
 1   label_ham   5572 non-null   uint8 
 2   label_spam  5572 non-null   uint8 
dtypes: object(1), uint8(2)
memory usage: 54.5+ KB


In [10]:
data['Count'] = 0   # we are creating a column with name Count
for i in np.arange(0, len(data.Message)):
    data.loc[i, 'Count'] = len(data.loc[i, 'Message'])

In [11]:
data.head(10)

,Message,label_ham,label_spam,Count
0,"Go until jurong point, crazy.. Available only ...",1,0,111
1,Ok lar... Joking wif u oni...,1,0,29
2,Free entry in 2 a wkly comp to win FA Cup fina...,0,1,155
3,U dun say so early hor... U c already then say...,1,0,49
4,"Nah I don't think he goes to usf, he lives aro...",1,0,61
5,FreeMsg Hey there darling it's been 3 week's n...,0,1,148
6,Even my brother is not like to speak with me. ...,1,0,77
7,As per your request 'Melle Melle (Oru Minnamin...,1,0,160
8,WINNER!! As a valued network customer you have...,0,1,158
9,Had your mobile 11 months or more? U R entitle...,0,1,154


In [12]:
data.describe()
# describe is only for the numeric data

,label_ham,label_spam,Count
count,5572.000000,5572.000000,5572.000000
mean,0.865937,0.134063,80.118808
std,0.340751,0.340751,59.690841
min,0.000000,0.000000,2.000000
25%,1.000000,0.000000,36.000000
50%,1.000000,0.000000,61.000000
75%,1.000000,0.000000,121.000000
max,1.000000,1.000000,910.000000


Prcoessing Message

In [13]:
# looking for first two original messages
data['Message'][0]

'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [14]:
data['Message'][1]

'Ok lar... Joking wif u oni...'

Preparing WordVector Corpus

In [15]:
corpus = []

In [16]:
import nltk
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer
import re
# for stop word removal
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [17]:
ps = PorterStemmer()


In [18]:
for i in range(0, 5572):
    # regular expression, for which we have imported re previously
    msg = data['Message'][i]
    # deal with email address
    msg = re.sub('\b[\w\-.]+?@\w+?\.\w{2,4}\b', 'emailaddr', msg)
    # deal with URl
    msg = re.sub('(http[s]?\S+)|(\w+\.[A-Za-z]{2,4}\S*)', 'httpaddr', msg)
    # deal with money
    msg = re.sub('£|\$', 'moneysymb', msg)
    # deal with phone_numbers
    msg = re.sub('\b(\+\d{1,2}\s)?\d[\-(.]?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4}\b', 'phonenumbr', msg)
    # deal with numbers
    msg = re.sub('\d+(\.\d+)?', 'numbr', msg)
    
    # removing puunctuation marks
    msg = re.sub('[^\w\d\s]', ' ', msg)  
    
    
    if i<2 :
        print("\t\t\t\t\t Message", i)
    if i<2 :
        print("\n Regular expression - Message ", i, " : ", msg)
    
    msg = msg.lower()
    
    if i<2 :
        print("\n After lower case - Message", i, " : ", msg)
        
    msg = msg.split()   # after this step the msg will be in list form, bcoz we have applied the split function
    if i<2 :
        print("\n After splitting - Message", i, " : ", msg)
        
    # we need to remove the stop word, using the above libraries for stop word removal
    # we will seperate out the stop words and apply stemming techniques
    msg = [ps.stem(word) for word in msg if not word in set(stopwords.words('english'))]
    
    if i<2 :
        print("\n After stemming and stopword removal - Message", i, " : ", msg)
        
    msg = ' '.join(msg)
    
    if i<2 :
        print("\n Final Message", i, " : ", msg, "\n\n\n")
        
    corpus.append(msg)
    

					 Message 0

 Regular expression - Message  0  :  Go until jurong point  crazy   Available only in bugis n great world la e buffet    Cine there got amore wat   

 After lower case - Message 0  :  go until jurong point  crazy   available only in bugis n great world la e buffet    cine there got amore wat   

 After splitting - Message 0  :  ['go', 'until', 'jurong', 'point', 'crazy', 'available', 'only', 'in', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet', 'cine', 'there', 'got', 'amore', 'wat']

 After stemming and stopword removal - Message 0  :  ['go', 'jurong', 'point', 'crazi', 'avail', 'bugi', 'n', 'great', 'world', 'la', 'e', 'buffet', 'cine', 'got', 'amor', 'wat']

 Final Message 0  :  go jurong point crazi avail bugi n great world la e buffet cine got amor wat 



					 Message 1

 Regular expression - Message  1  :  Ok lar    Joking wif u oni   

 After lower case - Message 1  :  ok lar    joking wif u oni   

 After splitting - Message 1  :  ['ok', 'lar', 'joking',

Preparing vector for each message

In [19]:
cv = CountVectorizer()
cv

CountVectorizer()

In [20]:
data_input = cv.fit_transform(corpus).toarray()

In [21]:
data_input[0] # this is not an array not data frame hence cant use head(), they are also numeric form 

array([0, 0, 0, ..., 0, 0, 0])

In [22]:
data_input[1]

array([0, 0, 0, ..., 0, 0, 0])

Applying Classification:           
    1. input = prepared sparse matrix / vectors for each message          
    2. output = label i.e. Spam or Ham

In [23]:
data.head()

,Message,label_ham,label_spam,Count
0,"Go until jurong point, crazy.. Available only ...",1,0,111
1,Ok lar... Joking wif u oni...,1,0,29
2,Free entry in 2 a wkly comp to win FA Cup fina...,0,1,155
3,U dun say so early hor... U c already then say...,1,0,49
4,"Nah I don't think he goes to usf, he lives aro...",1,0,61


In [24]:
# using labels we prepare teh data output
data_output = data['label_ham']
print("Data output counts")
data_output.value_counts()

Data output counts


1    4825
0     747
Name: label_ham, dtype: int64

In [25]:
data['label_ham']

0       1
1       1
2       0
3       1
4       1
       ..
5567    0
5568    1
5569    1
5570    1
5571    1
Name: label_ham, Length: 5572, dtype: uint8

Splitting the data for training and testing 

In [26]:
from sklearn.model_selection import train_test_split 

In [30]:
train_x, test_x, train_y, test_y = train_test_split(data_input, data_output, test_size = 0.20, random_state=0 )


Applying the ML models                                                   
1. RandomForest CLassifier 
2. Gaussian Naive Bayes
3. Decision Tree

In [31]:
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

Training

In [32]:
model_nvb = GaussianNB()
model_nvb.fit(train_x, train_y)

model_dt = tree.DecisionTreeClassifier()
model_dt.fit(train_x, train_y)

model_rf = RandomForestClassifier(n_estimators = 100)
model_rf.fit(train_x, train_y)


RandomForestClassifier()

Prediction

In [33]:
prediction_nvb = model_nvb.predict(test_x)
prediction_dt = model_dt.predict(test_x)
prediction_rf = model_rf.predict(test_x)

Result Naive Bayes

In [45]:
print("Accuracy : % 0.5f \n\n" % accuracy_score(test_y, prediction_nvb))   # the %0.5f is important to write else there is error
# printing the calssification report
print("Calssification report : \n", classification_report (test_y, prediction_nvb))

Accuracy :  0.86368 


Calssification report : 
               precision    recall  f1-score   support

           0       0.53      0.85      0.65       166
           1       0.97      0.87      0.92       949

    accuracy                           0.86      1115
   macro avg       0.75      0.86      0.78      1115
weighted avg       0.90      0.86      0.88      1115



Result Decision Tree

In [46]:
print("Accuracy : % 0.5f \n\n" % accuracy_score(test_y, prediction_dt))   # the %0.5f is important to write else there is error
# printing the calssification report
print("Calssification report : \n", classification_report (test_y, prediction_dt))

Accuracy :  0.97130 


Calssification report : 
               precision    recall  f1-score   support

           0       0.92      0.88      0.90       166
           1       0.98      0.99      0.98       949

    accuracy                           0.97      1115
   macro avg       0.95      0.93      0.94      1115
weighted avg       0.97      0.97      0.97      1115



Result Random Forest

In [47]:
print("Accuracy : % 0.5f \n\n" % accuracy_score(test_y, prediction_rf))   # the %0.5f is important to write else there is error
# printing the calssification report
print("Calssification report : \n", classification_report (test_y, prediction_rf))

Accuracy :  0.97578 


Calssification report : 
               precision    recall  f1-score   support

           0       1.00      0.84      0.91       166
           1       0.97      1.00      0.99       949

    accuracy                           0.98      1115
   macro avg       0.99      0.92      0.95      1115
weighted avg       0.98      0.98      0.97      1115

